**Fun with optimzers**

Neural networks are generally trained with some form of gradient descent. Yet there are tons of flavors of "gradient-based optimization algorithms" - see this [fantastic blog post](http://ruder.io/optimizing-gradient-descent/), which will serve as a primary reference for this tutorial (Ruder 2016)

We'll cover mathematical logic, high-level execution in PyTorch, and lower-level implementation in PyTorch's C/C++ backend. Let's get into it!

Throughout this notebook, we'll refer to our parameters at $\theta$ and update them from time $t$ to time $t+1$. In other words, we'll perform some operation to update our parameters over one time step from $\theta_{t}$ to $\theta_{t+1}$

Each gradient descent optimizer begins by computing the partial derivative of the cost function $J(\theta)$ at time $t$ (i.e. $J(\theta_{t})$) with respect to each parameter. The *gradient* $\nabla$ is just the vectorized set of derivatives (scalar:vector :: derivative:gradient ?) We then scale the gradient by a learning rate $\eta$ and subtract the result from our $\theta_{t}$ to get our $\theta_{t+1}$

**Stochastic Gradient Descent**

The most straightforward implementation of gradient descent is (arguably) "batch" or "vanilla gradient descent." This computes the gradients using all of our data and then performs one parameter update. In math:

(1) $\theta_{t+1} = \theta_{t} - \eta \cdot J(\theta_{t})$

However, batch gradient descent doesn't work well with large datasets. Imagine if we were performing batch gradient descent on the ImageNet 2012 dataset (something like 138 GB). We'd have to load all 138 GB into memory and use the entire dataset to update our parameters.

Instead, what if we were to update parameters one example at a time? This is the idea behind *stochastic gradient descent*. In math:

(2) $\theta_{t+1} = \theta_{t} - \eta \cdot J(\theta_{t}; x^{(i)}; y^{(i)})$

The superscipted $(i)$ simply tells us to calculate our gradient and perform our updates after passing each example through the network.

Yet PyTorch implements neither pure batch nor pure stochastic gradient descent. Instead, it uses something called "mini-batch gradient descent." Rather than performing parameter updates over the entire dataset (batch) or one example at a time (stochastic), mini-batch gradient descent calculates the gradient and applies parameter updates to a "mini-batch" of *n* training examples:

(3) $\theta_{t+1} = \theta_{t} - \eta \cdot J(\theta_{t}; x^{(i:i+n)}; y^{(i:i+n)})$

Here the superscripted $(i:i+n)$ tells us to calculate our gradient and perform updates for $n$ observations $i$ through $i+n$, where $i$ is the first observation of our mini-batch.

PyTorch actually refers to mini-batch gradient descent as "stochastic gradient descent," which is kind of confusing for someone trying to wrap their head around optimizers.

The [documentation](https://pytorch.org/docs/stable/optim.html) explains the use of optimizers in PyTorch. `torch.optim.Optimizer` is the base class which contains our optimizer subclasses: `Adadelta`, `Adagrad`, `Adam`, `AdamW`, `SparseAdam`, `Adamax`, `ASGD`, `LBFGS`, `RMSprop`, `Rprop`, and `SGD`.

However, before doing any of this, we need to train a model. A relatively easy way to do this is to use the `mnist` network from the PyTorch `examples` repository. From the command line:

`git clone https://github.com/pytorch/examples.git`

The network is defined in `main.py`. Let's run the network from the command line for ten epochs and report the accuracy on the validation data:

`python main.py --epochs=10`

We get the following loss and accuracy on our test set after each epoch. These numbers were consistent across two runs of the script for the first epoch:

`Epoch 1: Average loss: 0.1017, Accuracy: 9669/10000 (97%)
Epoch 2: Average loss: 0.0614, Accuracy: 9828/10000 (98%)
Epoch 3: Average loss: 0.0562, Accuracy: 9809/10000 (98%)
Epoch 4: Average loss: 0.0409, Accuracy: 9864/10000 (99%)
Epoch 5: Average loss: 0.0384, Accuracy: 9873/10000 (99%)
Epoch 6: Average loss: 0.0336, Accuracy: 9893/10000 (99%)
Epoch 7: Average loss: 0.0343, Accuracy: 9876/10000 (99%)
Epoch 8: Average loss: 0.0391, Accuracy: 9878/10000 (99%)
Epoch 9: Average loss: 0.0288, Accuracy: 9909/10000 (99%)
Epoch 10: Average loss: 0.0314, Accuracy: 9895/10000 (99%)`

If we search for `optim`, we see that our `optimizer` is defined as follows:

`optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)`

Thus we're using (mini-batch) "stochastic" gradient descent. If we consult the documentation, we see that `optim.SGD` takes six arguments (data type in parentheses):

`params` (iterable): the parameters we're optimizing, in this case the parameters from our model trained on the MNIST dataset (a five-layer convolutional neural network that suspiciously resembles - but is not identical to - LeNet-5)

`lr` (float): our learning rate $\eta$. This is optionally passed as an argument when calling `python main.py`. If no argument is passed, `lr` defaults to `0.01`

`momentum` (float): We can optionally add momentum to accelerate gradient descent. [Sutskever et al. 2013](http://www.cs.toronto.edu/%7Ehinton/absps/momentum.pdf) includes the "classical" mathematical formalization of momentum:

(4) $v_{t+1} = \mu v_{t} - \epsilon \nabla f(\theta_{t})$ <br/>
(5) $\theta_{t+1} = \theta_{t} + v_{t+1}$

Let's unpack this. $\nabla f(\theta_{t})$ is just the gradient of our loss function at time $t$ (the formalization here uses $f(t)$ instead of $J(t)$). Similarly, $\epsilon$ is our learning rate, equivalent to $\eta$ in our formulations.

Rather than simply subtracting our scaled (by the learning rate) gradient from $\theta_{t}$ to get $\theta_{t+1}$, we now introduce a momentum term $v$, which we similarly update from time $t$ to time $t+1$. We scale $v_{t}$ by $\mu$ and update the result in the same manner as we update our parameters using gradient descent:

(6) $v_{t+1} = \mu v_{t} - \epsilon \nabla f(\theta_{t})$

To update our parameters to $\theta_{t+1}$, we simply add our updated momentum term to $\theta_{t}$:

(7) $\theta_{t+1} = \theta_{t} + v_{t+1}$

By default, PyTorch implements `SGD` without momentum (`momentum=0`)

`dampening` isn't explained in the documentation. Someone [campaigned for its removal](https://github.com/pytorch/pytorch/issues/6) a while back, but the PyTorch overlords decided to set it to 0 instead. Let's not worry about it for now - it's only an argument for `SGD` - we can examine its implementation when we dive into the C/C++ backend.

`weight_decay` adds a L2 regularization term (again, this is 0 by default, and we'll worry about implementation when we get to the backend).

`nesterov` computes updates using Nesterov's Acclerated Gradient if set to `True` (again, it's `False` by default). All this means is that we add our scaled momentum term at time $t$ to our parameters before computing our gradients. Once we've updated our momentum, the parameter update is identical:

(8) $v_{t+1} = \mu v_{t} - \epsilon \nabla f(\theta_{t} + \mu v_{t})$

(9) $\theta_{t+1} = \theta_{t} + v_{t+1}$

**Adagrad**

Next up, we'll examine the Adagrad optimization algorithm. Instead of using a fixed learning rate, Adagrad "**ada**pts the learning rate to the parameters... smaller updates (lower learning rates) for parameters associated with frequently occurring features, and larger updates (higher learning rates) for parameters associated with infrequent features" (Ruder 2016). 

How does this work? We need to define different learning rates for different parameters. Recall that our gradient at time $t$ is defined as $\nabla J(\theta_{t})$ (or $\nabla f(\theta_{t})$ if you prefer the more general function symbol $f(t)$ to the more specific loss function symbol $J(t)$). We can specify that we are computing the gradient for parameters $\theta$ by rewriting the gradient as $\nabla_{\theta} J(\theta_{t})$

Now let's say we want to specify the gradient further, to represent a single parameter $\theta_{i}$ rather than the entire set of parameters $\theta$. At a single point in time, our set of parameters becomes $\theta_{t}$ (easily represented as a vector) and our single parameter becomes $\theta_{t,i}$ (which we can represent as an element of the vector $\theta_{t}$). Our gradient $g$ for this single parameter $i$ at time $t$ is thus $\eta_{t,i} = \nabla_{\theta} J(\theta_{t,i})$

Using this definition $g_{i,t}$ of our gradient $g$ for parameter $i$ at time $t$, we can redefine (mini-batch) stochastic gradient descent as performing the following update:

(10) $\theta_{t+1,i} = \theta_{t,i} - \eta \cdot \eta_{t,i}$

So where does Adagrad come in? In place of the general learning rate $\eta$, we now have this term:

(11) $\frac{\eta}{\sqrt{G_{t,(i,i)} + \epsilon}}$

Here $G_{t}$ is a diagonal matrix (all the entries are zero except for those on the main diagonal). The elements of $G_{t}$ are denoted by $i,i$, and each of these "is the sum of the squares of the gradients with respect to $\theta_{i}$ up to time step $t$ (Ruder 2016). We add $\epsilon$ so that our denominator cannot equal zero (thus making the term undefined). Putting it all together, we have the following rule to update parameters with Adagrad:

(12) $\theta_{t+1,i} = \theta_{t,i} - \frac{\eta}{\sqrt{G_{t,(i,i)} + \epsilon}} \cdot \eta_{t,i}$

We can then extend Adagrad to all parameters by taking the product of our modified diagonal matrix and our vector of parameters:

(13) $\theta_{t+1} = \theta_{t} - \frac{\eta}{\sqrt{G_{t} + \epsilon}} \cdot \eta_{t}$

Now let's implement Adagrad in PyTorch. `torch.optim.Adagrad` takes five arguments:

`params`: same as `SGD`.

`lr`: same as `SGD`. The default `lr` for Adagrad is 0.01, identical to that in `examples/mnist/main.py`, so let's use that value.

`lr_decay`: We can also decrease our learning rate by a specified factor after a specified number of epochs. It isn't clear how PyTorch implements this, but we're going to stick with the default value of `0` anyway, so let's not worry about it for now.

`weight_decay`: same as `SGD` (default also `0`).

`eps`: this is our $\epsilon$ - we'll use the (tiny) default value of 1e-10.

What happens when we swap in `torch.optim.Adagrad` as our optimizer (making sure to remove the `momentum` argument, which is not present in `Adagrad`)? Our `optimizer` now equals `optim.Adagrad(model.parameters())` - or `optim.Adagrad(model.parameters(), lr=args.lr)` if we explicitly pass the learning rate as `args.lr`. However, `args.lr` is set to the default value of `0.01`, so we don't need to include (or exclude) this argument.

Let's compare accuracy with `SGD` (not worrying for now about training speed):

`Epoch    Loss (SGD)    Loss (Adagrad)   Acc (SGD)   Acc (Adagrad)
 1        0.1017        0.0479           0.9669      0.9843
 2        0.0614        0.0349           0.9828      0.9888
 3        0.0562        0.0315           0.9809      0.9897
 4        0.0409        0.0291           0.9864      0.9908
 5        0.0384        0.0285           0.9873      0.9907
 6        0.0336        0.0251           0.9893      0.9912
 7        0.0343        0.0253           0.9876      0.9920
 8        0.0391        0.0306           0.9878      0.9899
 9        0.0288        0.0266           0.9909      0.9922
 10       0.0314        0.0294           0.9895      0.9897`
 
What we see is that Adagrad tends to converge faster (in terms of the number of epochs necessary to train to e.g. 99% accuracy). However, by 10 epochs the gap between SGD and Adagrad has decreased.

**Adadelta**

Whereas Adagrad stores all prior squared gradients, Adadelta limits the number of accumulated gradients to a constant $w$. We do this by first defining our "sum of gradients... recursively... as a decaying average of all past squared gradients" (Ruder 2016). The output of this accumulation is a "running average" $E[g^{2}]_{t}$, which is computed as a weighted sum of its value at the prior time step $E[g^{2}]_{t-1}$ and the value of the gradient at the current time $g^{2}_{t}$. The relative weight of the prior average and current gradient is defined by $\gamma$ such that:

(14) $E[g^{2}]_{t} = \gamma[g^{2}]_{t-1} + (1-\gamma)g^{2}_{t}$

Note that PyTorch uses the parameter `rho` ($\rho$) rather than gamma ($\gamma$) to weight the prior average relative to the current gradient. Ruder suggests 0.9 for this (momentum) term, and PyTorch implements this value by default in `torch.optim.Adadelta`

Now let's define a "parameter update vector" $\Delta \theta_{t}$ as the change in our gradient at time $t$. Again, this is simply the gradient scaled (negatively) by the learning rate (since we are subtracting our scaled gradient from the prior parameter values). This yields the following expression:

(15) $\Delta \theta_{t} = -\eta \cdot g_{t,i}$

To get our updated parameter vector, we simply add our parameter update vector to our current parameter vector (if we're updating from $t$ to $t+1$. If we're updating from $t-1$ to $t$, we'd be updating from the prior to current parameter vector):

(16) $\theta_{t+1} = \theta_{t} + \Delta \theta_{t}$

We can use our definition of $\Delta \theta_{t}$ in (15) to define the parameter update vector in Adagrad as follows:

(17) $\Delta \theta_{t} = - \frac{\eta}{\sqrt{G_{t} + \epsilon}} \cdot g_{t}$

We then swap in our running average $E[g^{2}]_{t}$ for Adagrad's diagonal matrix of summed square gradients $G_{t}$:

(18) $\Delta \theta_{t} = - \frac{\eta}{\sqrt{E[g^{2}]_{t} + \epsilon}} \cdot g_{t}$

Since our denominator is equivalent to the root mean squared (RMS) error:

(19) $\Delta \theta_{t} = - \frac{\eta}{RMS[g]_{t}} \cdot g_{t}$

Ruder (2016), citing the author of the Adadelta paper (Zeiler 2012), points out that there's a mismatch between the (hypothetica) unites of the parameter update $\Delta \theta_{t}$ and the original parameter vector $\theta_{t}$. To solve this problem, Zeiler constructs an "exponentially decaying average" analogous to our momentum term in (14) but with squared parameter updates $\Delta [\theta^{2}]_{t}$:

(20) $E[\Delta \theta^{2}]_{t} = \gamma E[\Delta \theta^{2}]_{t-1} + (1-\gamma) \Delta \theta^{2}_{t}$

As with (19), we can redefine (20) as the RMS error of parameter updates:

(21) $RMS[\Delta \theta]_{t} = \sqrt{E[\theta^{2}]_{t} + \epsilon}$

Since we don't actually know the RMS error of parameter updates at time $t$, we use the RMS error of parameter updates up to the prior time $t-1$. Swapping the RMS error up to $t-1$ for our learning rate in (19) gives us our parameter update vector for Adadelta:

(22) $\theta_{t} = - \frac{RMS[\Delta \theta]_{t-1}}{RMS[g]_{t}} \cdot g_{t}$

And again, we get our updated parameter vector by adding our parameter update vector to our current parameter vector:

(23) $\theta_{t+1} = \theta_{t} + \Delta \theta_{t}$

Let's go ahead and implement Adadelta in PyTorch. `torch.optim.Adadelta` takes the following arguments:

`params`: our model parameters (same as in `SGD` and `Adagrad`).

`rho`: momentum term by which we weight our current and squared gradients when computing the running average. We'll use the default value 0.9 - no need to pass a value explicitly to `optim.Adadelta`.

`eps`: epsilon value added to denominator. Again, we'll accept the default value 1e-6.

`lr`: our learning rate, which starts at 1.0 before adapting to parameters - we'll accept this default.

`weight_decay`: optional L2 penalty - we'll accept the default of zero penalty.

Let's now swap in `Adadelta` so that `optimizer = optim.Adadelta(model.parameters())` (making sure to remove `lr=args.lr` if it's still there, or at least (re-)setting `lr` to its default value of `1.0` for `Adadelta`. We'll run for 10 epochs again, recording our loss and accuracy for the test set after each epoch and comparing it to our results for `SGD` and `Adagrad`:

`Ep / Loss: SGD / Adagrad / Adadelta  Acc: SGD / Adagrad / Adadelta
 1    .1017      .0479      .0315     .9669     .9843      .9901
 2    .0614      .0349      .0280     .9828     .9888      .9902
 3    .0562      .0315      .0295     .9809     .9897      .9912
 4    .0409      .0291      .0345     .9864     .9908      .9905
 5    .0384      .0285      .0364     .9873     .9907      .9918
 6    .0336      .0251      .0276     .9893     .9912      .9924
 7    .0343      .0253      .0360     .9876     .9920      .9922
 8    .0391      .0306      .0289     .9878     .9899      .9929
 9    .0288      .0266      .0323     .9909     .9922      .9933
 10   .0314      .0294      .0366     .9895     .9897      .9931`
 
Adadelta appears to converge to higher accuracy quicker than Adagrad, which in turn converges quicker than SGD. While it generally has the highest accuracy, its loss is slightly higher than Adagrad's for five of the seven epochs between four and 10 (inclusive).

**RMSprop**

RMSprop is simply the first stage of the Adadelta update, as defined in (14):

(24) $E[g^{2}]_{t} = \gamma[g^{2}]_{t-1} + (1-\gamma)g^{2}_{t}$

We then swap this into our gradient update function (18), generating the following:

(25) $\theta_{t+1} = \theta_{t} - \frac{\eta}{\sqrt{E[g^{2}]_{t} + \epsilon}} \cdot g_{t}$

While the mathematical implementation of RMSprop is straightforward once we know how to implement Adadelta, `torch.optim.RMSprop` actually takes seven arguments. We already know `params`, `lr` (default: 0.01), `eps` ($\epsilon$, default: 1e-8), and `weight_decay` (default: 0). `centered` is a boolean variable (default: `False`) that "normalizes the gradient by an estimation of its variance" if `True` (the `torch.optim` documentation does not say how this is done.)

As for the other two arguments, the PyTorch documentation defines the "effective learning rate" as $\alpha / (\sqrt{v} + \epsilon)$. This appears to refer to the coefficient on the gradient $g_{t}$, where $\alpha$ (`alpha`) is our initial learning rate $\eta$, $v$ is our "weighted moving average of the squared gradient," and $\epsilon$ is `epsilon`. The remaining argument `momentum` is (apparently) the $\gamma$ from (24), but is set to 0 by default.

Let's now train `main.py` for 10 epochs using `torch.optim.RMSprop` with default arguments, recording our model loss and accuracy on the test set:

Loss:

`Ep    SGD       Adagrad    Adadelta   RMSprop
 1    .1017      .0479      .0315      .2838
 2    .0614      .0349      .0280      .1506
 3    .0562      .0315      .0295      .1037
 4    .0409      .0291      .0345      .1589
 5    .0384      .0285      .0364      .0973
 6    .0336      .0251      .0276      .1391
 7    .0343      .0253      .0360      .3066
 8    .0391      .0306      .0289      .2229
 9    .0288      .0266      .0323      .1393
 10   .0314      .0294      .0366      .1693`

Accuracy:

`Ep    SGD       Adagrad    Adadelta  RMSprop
 1    .9669      .9843      .9901     .9172
 2    .9828      .9888      .9902     .9592
 3    .9809      .9897      .9912     .9733
 4    .9864      .9908      .9905     .9740
 5    .9873      .9907      .9918     .9756
 6    .9893      .9912      .9924     .9678
 7    .9876      .9920      .9922     .9484
 8    .9878      .9899      .9929     .9491
 9    .9909      .9922      .9933     .9693
 10   .9895      .9897      .9931     .9686`
 
Yikes. Not very accurate, at least with default values.

**Adam**

The Adaptive Moment Estimation (Adam) optimizer computes both an exponentially decaying average of prior gradients (first moment) and prior squared gradients (second moment, like Adadelta and RMSprop):

(26) $m_{t} = \beta_{1} m_{t-1} + (1 - \beta_{1}) g_{t}$

(27) $v_{t} = \beta_{2} v_{t-1} + (1 - \beta_{2}) g^{2}_{t}$

However, when we initialize $m_{t}$ and $v_{t}$ as zero vectors, both terms bias towards zero early in the training process and when $\beta_{1}$ and $\beta_{2}$ are close to 1 (small decay rate). The Adam optimizer corrects for this by dividing the momentum terms by their decay rates:

(28) $\hat m_{t} = \frac{m_{t}}{1 - \beta^{t}_{1}}$

(29) $\hat v_{t} = \frac{v_{t}}{1 - \beta^{t}_{2}}$

Once we've computed $\hat m_{t}$ and $\hat v_{t}$, we can plug in the former term in place of our gradient in (25). As one might expect, the initial learning rate is divided by the square root of the latter term. However, our epsilon is outside of the root:

(30) $\theta_{t+1} = \theta_{t} - \frac{\eta}{\sqrt{^v_{t}}+\epsilon} \hat m_{t}$

Let's train our network using Adam with default argument values. We also have the option to use `amsgrad` (default: False), which we'll get to next.

Loss:

`Ep    SGD      Adagrad   Adadelta  RMSprop    Adam
 1    .1017     .0479     .0315     .2838     .0368
 2    .0614     .0349     .0280     .1506     .0358
 3    .0562     .0315     .0295     .1037     .0289
 4    .0409     .0291     .0345     .1589     .0309
 5    .0384     .0285     .0364     .0973     .0275
 6    .0336     .0251     .0276     .1391     .0255
 7    .0343     .0253     .0360     .3066     .0398
 8    .0391     .0306     .0289     .2229     .0292
 9    .0288     .0266     .0323     .1393     .0324
 10   .0314     .0294     .0366     .1693     .0442`

Accuracy:

`Ep    SGD      Adagrad   Adadelta  RMSprop    Adam
 1    .9669     .9843     .9901     .9172     .9876
 2    .9828     .9888     .9902     .9592     .9880
 3    .9809     .9897     .9912     .9733     .9990
 4    .9864     .9908     .9905     .9740     .9905
 5    .9873     .9907     .9918     .9756     .9920
 6    .9893     .9912     .9924     .9678     .9930
 7    .9876     .9920     .9922     .9484     .9898
 8    .9878     .9899     .9929     .9491     .9913
 9    .9909     .9922     .9933     .9693     .9917
 10   .9895     .9897     .9931     .9686     .9903`
 
Adam thus performs similarly to Adagrad and Adadelta.

**AMSGrad**

AMSGrad introduces an additional step to Adam (and the `AdamW` variant, which we'll get to next):

(31) $\hat v_{t} = max(\hat v_{t-1}, v_{t})$

We then pass $\hat v_{t}$ to our gradient update step. Let's compare loss and accuracy for `Adam` with `amsgrad=true` to our other optimizers:

Loss:

`Ep    SGD      Adagrad   Adadelta  RMSprop    Adam     Adam(AMSGrad)
 1    .1017     .0479     .0315     .2838     .0368     .0364
 2    .0614     .0349     .0280     .1506     .0358     .0361
 3    .0562     .0315     .0295     .1037     .0289     .0289
 4    .0409     .0291     .0345     .1589     .0309     .0295
 5    .0384     .0285     .0364     .0973     .0275     .0262
 6    .0336     .0251     .0276     .1391     .0255     .0226
 7    .0343     .0253     .0360     .3066     .0398     .0237
 8    .0391     .0306     .0289     .2229     .0292     .0295
 9    .0288     .0266     .0323     .1393     .0324     .0280
 10   .0314     .0294     .0366     .1693     .0442     .0291`

Accuracy:

`Ep    SGD      Adagrad   Adadelta  RMSprop    Adam     Adam(AMSGrad)
 1    .9669     .9843     .9901     .9172     .9876     .9881
 2    .9828     .9888     .9902     .9592     .9880     .9885
 3    .9809     .9897     .9912     .9733     .9990     .9898
 4    .9864     .9908     .9905     .9740     .9905     .9918
 5    .9873     .9907     .9918     .9756     .9920     .9917
 6    .9893     .9912     .9924     .9678     .9930     .9928
 7    .9876     .9920     .9922     .9484     .9898     .9928
 8    .9878     .9899     .9929     .9491     .9913     .9912
 9    .9909     .9922     .9933     .9693     .9917     .9915
 10   .9895     .9897     .9931     .9686     .9903     .9929`
 
Loss and accuracy for Adam is thus similar with or without AMSGrad. We'll remove this from future comparisons, sticking to the default arguments for each optimizer.

**AdamW**

This one isn't explained in Ruder (2016), but there's a paper that goes into AdamW's modifications to Adam (Loschilov & Hutter 2019). AdamW "decouples the optimal choice of weight decay factor from the setting of the learning rate." The paper goes into what this means exactly. Let's see if it works on MNIST with our simple CNN:

Loss:

`Ep    SGD      Adagrad   Adadelta  RMSprop    Adam     AdamW
 1    .1017     .0479     .0315     .2838     .0368     .0380
 2    .0614     .0349     .0280     .1506     .0358     .0320
 3    .0562     .0315     .0295     .1037     .0289     .0283
 4    .0409     .0291     .0345     .1589     .0309     .0267
 5    .0384     .0285     .0364     .0973     .0275     .0278
 6    .0336     .0251     .0276     .1391     .0255     .0399
 7    .0343     .0253     .0360     .3066     .0398     .0372
 8    .0391     .0306     .0289     .2229     .0292     .0344
 9    .0288     .0266     .0323     .1393     .0324     .0389
 10   .0314     .0294     .0366     .1693     .0442     .0346`

Accuracy:

`Ep    SGD      Adagrad   Adadelta  RMSprop    Adam     AdamW
 1    .9669     .9843     .9901     .9172     .9876     .9871
 2    .9828     .9888     .9902     .9592     .9880     .9905
 3    .9809     .9897     .9912     .9733     .9990     .9913
 4    .9864     .9908     .9905     .9740     .9905     .9925
 5    .9873     .9907     .9918     .9756     .9920     .9912
 6    .9893     .9912     .9924     .9678     .9930     .9891
 7    .9876     .9920     .9922     .9484     .9898     .9915
 8    .9878     .9899     .9929     .9491     .9913     .9910
 9    .9909     .9922     .9933     .9693     .9917     .9910
 10   .9895     .9897     .9931     .9686     .9903     .9918`
 
There doesn't seem to be much differentiating our four `Ada` optimizers. Maybe we need a tougher classification problem, such as CIFAR, or maybe another task altogether?

**AdaMax**

Recall our formula for computing our second moment in Adam. We can rewrite $g_{t}^{2}$ as the equivalent $|g_{t}|^{2}$:

(32) $v_{t} = \beta_{2} v_{t-1} + (1 - \beta_{2}) |g_{t}|^{2}$

Then we can generalize our updates to any degree of moment ($l_{p}$ norm):

(33) $v_{t} = \beta^{p}_{2} v_{t-1} + (1 - \beta^{p}_{2}) |g_{t}|^{p}$

As $p$ increases without bound, $v_{t}$ converges to the following value, denoted by $u_{t}$:

(34) $u_{t} = \beta^{\infty}_{2} v_{t-1} + (1 - \beta^{\infty}_{2}) |g_{t}|^{\infty} = max(\beta_{2} \cdot v_{t-1}, |g_{t}|)$

Swapping this into (30), we have our parameter update rule for AdaMax:

(35) $\theta_{t-1} = \theta_{t} - \frac{\eta}{u_{t}} \hat m_{t}$

Now let's test loss and accuracy for `torch.optim.Adamax` with defaults:

Loss:

`Ep   SGD     Adagrad  Adadelta RMSprop   Adam    AdamW    Adamax
 1   .1017    .0479    .0315    .2838    .0368    .0380    .0422
 2   .0614    .0349    .0280    .1506    .0358    .0320    .0382
 3   .0562    .0315    .0295    .1037    .0289    .0283    .0348
 4   .0409    .0291    .0345    .1589    .0309    .0267    .0265
 5   .0384    .0285    .0364    .0973    .0275    .0278    .0217
 6   .0336    .0251    .0276    .1391    .0255    .0399    .0244
 7   .0343    .0253    .0360    .3066    .0398    .0372    .0246
 8   .0391    .0306    .0289    .2229    .0292    .0344    .0246
 9   .0288    .0266    .0323    .1393    .0324    .0389    .0249
 10  .0314    .0294    .0366    .1693    .0442    .0346    .0337`

Accuracy:

`Ep   SGD     Adagrad  Adadelta RMSprop   Adam    AdamW    Adamax
 1   .9669    .9843    .9901    .9172    .9876    .9871    .9869
 2   .9828    .9888    .9902    .9592    .9880    .9905    .9874
 3   .9809    .9897    .9912    .9733    .9990    .9913    .9871
 4   .9864    .9908    .9905    .9740    .9905    .9925    .9912
 5   .9873    .9907    .9918    .9756    .9920    .9912    .9933
 6   .9893    .9912    .9924    .9678    .9930    .9891    .9923
 7   .9876    .9920    .9922    .9484    .9898    .9915    .9928
 8   .9878    .9899    .9929    .9491    .9913    .9910    .9919
 9   .9909    .9922    .9933    .9693    .9917    .9910    .9930
 10  .9895    .9897    .9931    .9686    .9903    .9918    .9907`
 
This is about as good as we've done.

**SparseAdam**

This is a "lazy version of Adam suitable for sparse tensors" (Ruder 2016). We only perform updates on moments "that show up in the gradient." If we try to run `torch.optim.SparseAdam` on `main.py`, we get the following error message:

`RuntimeError: SparseAdam does not support dense gradients, please consider Adam instead`

Ok, on to the next one.

**ASGD**

`torch.optim.ASGD` applies Averaged Stochastic Gradient Descent. According to [Wikipedia](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Averaging), ASGD "records an average of its parameter vector over time." We then use this averaged parameter vector to update our parameters. Let's see how it performs:

Loss:

`Ep  SGD    Adagrad Adadelta RMSprop  Adam   AdamW   Adamax  ASGD
 1  .1017   .0479   .0315    .2838   .0368   .0380   .0422  .1680
 2  .0614   .0349   .0280    .1506   .0358   .0320   .0382  .1000
 3  .0562   .0315   .0295    .1037   .0289   .0283   .0348  .0727
 4  .0409   .0291   .0345    .1589   .0309   .0267   .0265  .0586
 5  .0384   .0285   .0364    .0973   .0275   .0278   .0217  .0616
 6  .0336   .0251   .0276    .1391   .0255   .0399   .0244  .0451
 7  .0343   .0253   .0360    .3066   .0398   .0372   .0246  .0440
 8  .0391   .0306   .0289    .2229   .0292   .0344   .0246  .0457
 9  .0288   .0266   .0323    .1393   .0324   .0389   .0249  .0346
 10 .0314   .0294   .0366    .1693   .0442   .0346   .0337  .0392`

Accuracy:

`Ep  SGD    Adagrad Adadelta RMSprop  Adam   AdamW   Adamax  ASGD
 1  .9669   .9843   .9901    .9172   .9876   .9871   .9869  .9504
 2  .9828   .9888   .9902    .9592   .9880   .9905   .9874  .9711
 3  .9809   .9897   .9912    .9733   .9990   .9913   .9871  .9773
 4  .9864   .9908   .9905    .9740   .9905   .9925   .9912  .9819
 5  .9873   .9907   .9918    .9756   .9920   .9912   .9933  .9811
 6  .9893   .9912   .9924    .9678   .9930   .9891   .9923  .9852
 7  .9876   .9920   .9922    .9484   .9898   .9915   .9928  .9850
 8  .9878   .9899   .9929    .9491   .9913   .9910   .9919  .9848
 9  .9909   .9922   .9933    .9693   .9917   .9910   .9930  .9889
 10 .9895   .9897   .9931    .9686   .9903   .9918   .9907  .9866`

**LBFGS**

The L-BFGS optimizer, implemented with `torch.optim.LBFGS`, is "heavily inspired by *minFunc*," though it's unclear from the documentation alone what L-BFGS does specifically. `LBFGS` also requires this additional `closure` argument. Since the implementation is unclear, the optimizer isn't covered in Ruder (2016), and the script won't run without the `closure` argument in `step()`, we'll skip this one.

**Rprop**

Rprop is the "resilient backpropagation algorithm." Ruder (2016) doesn't mention Rprop, and the documentation doesn't explain its implementation. We get pretty terrible average loss after one epoch (14.2588), with accuracy of .9050, and it doesn't even appear that the loss is converging, so let's also skip this one.

That covers the optimizers in PyTorch! For MNIST, it looks like we're good with any of the momentum-based optimizers. It'd be interesting to compare accuracy and loss across more complex classification problems (e.g. RGB, 100 or 1000 classes) and other tasks.